- This Jupyter notebook is an example of the how to access the RestAPI interface, described at:
  https://doc.particle.one/

# Credentials / Settings

In [9]:
# Make sure you have `pandas` installed.
# If you don't have it installed, install it in your usual way (with pip, conda)

#! pip install requests pandas

import json

import os

import pandas as pd
import requests

In [10]:
# Enter your token here.
# You can get your token by signing up at `www.particle.one`.
#TOKEN = "YOUR_TOKEN_HERE"
# An example token is like:
TOKEN = "e44e7c6b04ef3ea1cfb7a8a67db74751c177259e"

API_URL = "https://data.particle.one"
HEADERS = {"Authorization": f"Token {TOKEN}", "Content-Type": "application/json"}

# Search query structure

Search query is a Python `dict` with the following structure:
```python
query = {
    "text": "",
    "commodity": [],
    "business_category": "",
    "country": [],
    "frequency": []
}
```
The fields are:
- `text`: string. Works as a filter. Free text. Everything that have no match with this phrase will be filtered out.
- `commodity`: list of strings. Works as a filter. You can find valid values in paragraph 7.1 of this notebook.
- `business_category`: string. Works as a filter. You can find valid values in paragraph 7.2 of this notebook.
- `country`: list of strings. Works as a filter. You can find valid values in paragraph 7.3 of this notebook.
- `frequency`: list of strings. Works as a filter. You can find valid values in paragraph 7.4 of this notebook.

Combination of fields work with logical operator AND.
E.g. you will get all records that satisfy all filters.

`text` **AND** `commodity` **AND** `business_category` **AND** `country` **AND** `frequency`

# Imports

# POST data-api/v1/search-count/
Returns count for the given query.

In [11]:
# Build entrypoint url.
count_url = os.path.join(API_URL, "data-api/v1/search-count/")
print("count_url=", count_url)

count_url= https://data.particle.one/data-api/v1/search-count/


In [12]:
# Prepare query.
query = {
    "text": "",
    "commodity": ["Corn"],
    "business_category": "",
    "country": [],
    "frequency": [],
}
payload = json.dumps(query)

In [13]:
# Perform query.
response = requests.request("POST", count_url, headers=HEADERS, data=payload)
data = json.loads(response.text.encode("utf8"))
print("data=", data)

data= {'message': 'OK', 'count': 6}


# POST data-api/v1/search/

- It returns the first chunk of the payload metadata for the given query, where a chunk is 1000 records.
- It also returns `scroll_id` to get the next portion of the data.

In [14]:
search_url = os.path.join(API_URL, "data-api/v1/search/")
print("search_url=", search_url)

search_url= https://data.particle.one/data-api/v1/search/


In [15]:
# Prepare query.
query = {
    "text": "Gas",
    "commodity": [],
    "business_category": "",
    "country": [],
    "frequency": [],
}
payload = json.dumps(query)

In [16]:
# Perform query.
response = requests.request("POST", search_url, headers=HEADERS, data=payload)
data = json.loads(response.text.encode("utf8"))
print("data.keys()=", list(data.keys()))

assert "detail" not in data, data

print("total_count=", data["total_count"])

# Saving scroll_id for the next query.
scroll_id = data["scroll_id"]
print("scroll_id=", scroll_id)

df = pd.DataFrame.from_records(data["rows"])
print("df.shape=", df.shape)
print("df.head()=")
display(df.head())

data.keys()= ['message', 'scroll_id', 'total_count', 'rows']
total_count= 29545
scroll_id= DXF1ZXJ5QW5kRmV0Y2gBAAAAAABDhIkWQUFleVN5Ym9RT0daMWhMQU1KNXJ4dw==
df.shape= (1000, 8)
df.head()=


,name,commodity,payload_id,business_category,country,frequency,unit,start_date
0,Actual Market Prices: World: Natural Gas: I...,Liquefied Natural Gas,1632886ae0ee7c8e785769009c1a276f2abf256a,Upstream,Undefined,Monthly,USD/MBTU,1992-01-01
1,Actual Market Prices: World: Natural Gas: I...,Natural Gas,1632886ae0ee7c8e785769009c1a276f2abf256a,Upstream,Undefined,Monthly,USD/MBTU,1992-01-01
2,Actual Market Prices: World: Natural Gas: R...,Natural Gas,dded2f042b2c773790ececfb7565aa1db8b0992c,Upstream,Undefined,Monthly,USD/MBTU,1985-01-01
3,Actual Market Prices: World: Natural Gas: U...,Natural Gas,446c3a3dc964bb48bf88deaf069e59e52b3b057b,Upstream,Undefined,Monthly,USD/MBTU,1991-01-01
4,Alamo – TX Natural Gas Pipeline Exports to Mex...,Natural Gas,dff30995669368dcd485c632c0dccc8188f47fff,Undefined,Mexico,Annual,USD/thousand CF,None


# GET data-api/v1/search-scroll/?scroll_id=

In [17]:
# Build entrypoint url.

# We use scroll id from the previous query.
search_scroll_url = os.path.join(
    API_URL, f"data-api/v1/search-scroll/?scroll_id={scroll_id}"
)
print("search_scroll_url=", search_scroll_url)

search_scroll_url= https://data.particle.one/data-api/v1/search-scroll/?scroll_id=DXF1ZXJ5QW5kRmV0Y2gBAAAAAABDhIkWQUFleVN5Ym9RT0daMWhMQU1KNXJ4dw==


In [18]:
# Perform query.

response = requests.request("GET", search_scroll_url, headers=HEADERS)
data = json.loads(response.text.encode("utf8"))
print("data.keys()=", list(data.keys()))
print("data['rows'][0]=", data["rows"][0])

df = pd.DataFrame.from_records(data["rows"])
print("df.shape=", df.shape)
print("df.head()=")
display(df.head())

data.keys()= ['message', 'scroll_id', 'rows']
data['rows'][0]= {'name': 'Dry natural gas consumption – Barbados – billion cubic meters', 'commodity': 'Natural Gas', 'payload_id': '6eeb38e07d5a5c4b9c7b7d0ee06ba83eff7fb981', 'business_category': 'Undefined', 'country': 'Barbados', 'frequency': 'Annual', 'unit': 'kwh, in billions', 'start_date': None}
df.shape= (1000, 8)
df.head()=


,name,commodity,payload_id,business_category,country,frequency,unit,start_date
0,Dry natural gas consumption – Barbados – billi...,Natural Gas,6eeb38e07d5a5c4b9c7b7d0ee06ba83eff7fb981,Undefined,Barbados,Annual,"kwh, in billions",None
1,Dry natural gas consumption – Barbados – milli...,Natural Gas,05780468618459e59450219200206e8b17463050,Undefined,Barbados,Annual,"BTU/person, in millions",None
2,Dry natural gas consumption – Barbados – quadr...,Natural Gas,4f15dcf120eec1a39546c3b65e0f392ceb09495c,Undefined,Barbados,Annual,"BTU, in quadrillions",None
3,Dry natural gas consumption – Barbados – teraj...,Natural Gas,fab1d3dc9a3236a3abaeedf09b394788f0de21fa,Undefined,Barbados,Annual,terajoules,None
4,Dry natural gas consumption – Belarus – billio...,Natural Gas,5960c89357da83583b5dd2a85c59678fdc464e4e,Undefined,Belarus,Annual,"USD at PPP, in billions",None


# GET data-api/v1/payload/?payload_id=
Returns payload for the given `payload_id`

In [19]:
# Build entrypoint url.

# We use one of the `payload_id` from one of the previous queries.
payload_id = "8f26ba4734df3a62352cce9d64987d64da54b400"
payload_url = os.path.join(
    API_URL, f"data-api/v1/payload/?payload_id={payload_id}"
)
print("payload_url=", payload_url)

payload_url= https://data.particle.one/data-api/v1/payload/?payload_id=8f26ba4734df3a62352cce9d64987d64da54b400


In [20]:
# Perform query.
response = requests.request("GET", payload_url, headers=HEADERS)
data = json.loads(response.text.encode("utf8"))
print("data.keys()=", list(data.keys()))

df = pd.DataFrame.from_records(data["payload_data"])
print("df.shape=", df.shape)
print("df.head()=")
display(df.head())

data.keys()= ['message', 'payload_data']
df.shape= (48, 4)
df.head()=


,original_period,original_value,period,value
0,1980,0.0,1980-01-01T00:00:00,0
1,1981,0.0,1981-01-01T00:00:00,0
2,1982,0.0,1982-01-01T00:00:00,0
3,1983,0.0,1983-01-01T00:00:00,0
4,1984,0.0,1984-01-01T00:00:00,0


# Helpers

To search we use several predefined lists of names for each parameter of metadata. Such as:
- `commodity`
- `business-category`
- `country`
- `frequency`

Each parameter has its own set of valid names.

## GET data-api/v1/commodities/

In [21]:
# Build entrypoint url.
commodities_url = os.path.join(API_URL, "data-api/v1/commodities/")
print("commodities_url=", commodities_url)

commodities_url= https://data.particle.one/data-api/v1/commodities/


In [22]:
# Perform query.
response = requests.request("GET", commodities_url, headers=HEADERS)
data = json.loads(response.text.encode("utf8"))
print("data.keys()=", list(data.keys()))

df = pd.DataFrame.from_records(data["data"])
print("df.shape=", df.shape)
print("df.head()=")
display(df.head())

data.keys()= ['message', 'data']
df.shape= (61, 1)
df.head()=


,name
0,Aluminum
1,Benzene
2,Biodiesel
3,Biofuel
4,Butadiene


## GET data-api/v1/business-categories/

In [23]:
# Build entrypoint url.
bc_url = os.path.join(API_URL, "data-api/v1/business-categories/")
print("bc_url=", bc_url)

bc_url= https://data.particle.one/data-api/v1/business-categories/


In [24]:
# Perform query.

response = requests.request("GET", bc_url, headers=HEADERS)
data = json.loads(response.text.encode("utf8"))
print("data.keys()=", list(data.keys()))

df = pd.DataFrame.from_records(data["data"])
print("df.shape=", df.shape)
print("df.head()=")
display(df.head())

data.keys()= ['message', 'data']
df.shape= (4, 1)
df.head()=


,name
0,Downstream
1,Midstream
2,Undefined
3,Upstream


## GET data-api/v1/countries/

In [25]:
# Build entrypoint url.
countries_url = os.path.join(API_URL, "data-api/v1/countries/")
print("countries_url=", countries_url)

countries_url= https://data.particle.one/data-api/v1/countries/


In [26]:
# Perform query.
response = requests.request("GET", countries_url, headers=HEADERS)
data = json.loads(response.text.encode("utf8"))
print("data.keys()=", list(data.keys()))

df = pd.DataFrame.from_records(data["data"])
print("df.shape=", df.shape)
print("df.head()=")
display(df.head())

data.keys()= ['message', 'data']
df.shape= (231, 1)
df.head()=


,name
0,Afghanistan
1,Albania
2,Algeria
3,American Samoa
4,Andorra


## GET data-api/v1/frequencies/

In [27]:
# Build entrypoint url.
frequencies_url = os.path.join(API_URL, "data-api/v1/frequencies/")
print(frequencies_url)

https://data.particle.one/data-api/v1/frequencies/


In [28]:
# Perform query.
response = requests.request("GET", frequencies_url, headers=HEADERS)
data = json.loads(response.text.encode("utf8"))
print("data.keys()=", list(data.keys()))

df = pd.DataFrame.from_records(data["data"])
print("df.shape=", df.shape)
print("df.head()=")
display(df.head())

data.keys()= ['message', 'data']
df.shape= (6, 1)
df.head()=


,name
0,Daily
1,Weekly
2,Monthly
3,Quarterly
4,Semi-annual
